In [1]:
from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
import requests

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# Replace with your actual API key
API_URL="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/silchar/today?unitGroup=us&key=4RH3KGN9E38EDN8W7XJT42SQ5&contentType=json"
@app.get("/weather/")
def get_weather(location: str = Query(..., description="City or location name")):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/today?unitGroup=us&key=4RH3KGN9E38EDN8W7XJT42SQ5&contentType=json"


    response = requests.get(url)

    if response.status_code != 200:
        return {"error": "Failed to fetch weather data"}

    data = response.json()
    return data['days']


In [2]:
import pandas as pd
data=pd.DataFrame(get_weather(location="guwahati"))

In [3]:
data.shape

(1, 37)

In [4]:

# Convert datetime to features
data['datetime'] = pd.to_datetime(data['datetime'])
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['day_of_year'] = data['datetime'].dt.dayofyear

# Extract time components from sunrise and sunset
data['sunrise_hour'] = pd.to_datetime(data['sunrise']).dt.hour
data['sunrise_minute'] = pd.to_datetime(data['sunrise']).dt.minute
data['sunset_hour'] = pd.to_datetime(data['sunset']).dt.hour
data['sunset_minute'] = pd.to_datetime(data['sunset']).dt.minute
data['daylight_hours'] = (pd.to_datetime(data['sunset']) - pd.to_datetime(data['sunrise'])).dt.total_seconds()/3600

# Handle categorical variables (conditions)
conditions_dummies = pd.get_dummies(data['conditions'], prefix='cond')
data = pd.concat([data, conditions_dummies], axis=1)
# Ensure all expected condition columns are present
expected_conditions = [
        'cond_Clear', 'cond_Overcast',
       'cond_Partially cloudy', 'cond_Rain, Overcast',
       'cond_Rain, Partially cloudy'
    ]
for cond in expected_conditions:
        if cond not in data.columns:
            data[cond] = 0
data.rename(columns={'pressure': 'sealevelpressure'}, inplace=True)
# Drop unnecessary columns
drop_cols = ['datetimeEpoch','datetime', 'sunriseEpoch', 'sunsetEpoch','source', 'hours','sunrise', 'sunset', 'conditions', 'description', 
             'icon', 'stations', 'preciptype', 'severerisk']
data = data.drop(columns=drop_cols)

reindexing

In [5]:
correct_order = ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'moonphase', 'year', 'month', 'day',
       'day_of_year', 'sunrise_hour', 'sunrise_minute', 'sunset_hour',
       'sunset_minute', 'daylight_hours', 'cond_Clear', 'cond_Overcast',
       'cond_Partially cloudy', 'cond_Rain, Overcast',
       'cond_Rain, Partially cloudy']


# Reorder and align test data
data= data.reindex(columns=correct_order, fill_value=0)

In [6]:
data.shape

(1, 37)

In [7]:
data.columns

Index(['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'moonphase', 'year', 'month', 'day',
       'day_of_year', 'sunrise_hour', 'sunrise_minute', 'sunset_hour',
       'sunset_minute', 'daylight_hours', 'cond_Clear', 'cond_Overcast',
       'cond_Partially cloudy', 'cond_Rain, Overcast',
       'cond_Rain, Partially cloudy'],
      dtype='object')

In [8]:
# Handle missing values
data = data.fillna(data.mean())

In [9]:
# Define features and target
X = data.drop(columns=['solarenergy'])
y = data['solarenergy']

In [10]:
import pickle
with open("xgboost_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

X_scaled = scaler.transform(X)

<h4>FINAL PREDICTION</h4>

In [12]:
import joblib
with open("xgboost_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
model = joblib.load("best_xgb_model.pkl")
predictions = model.predict(X_scaled)
print(predictions)


[25.899761]


In [13]:
y

0    28.6
Name: solarenergy, dtype: float64